In [14]:
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output


In [20]:
# Load Verified Numbers
try:
    with open("scam_reports.json", "r") as f:
        verified_data = json.load(f)
except FileNotFoundError:
    verified_data = {}
    print("scam_reports.json not found. Initializing verified_data as empty.")


# Load or initialize Scam Reports
try:
    with open("scam_reports.json", "r") as f:
        scam_reports = json.load(f)
except FileNotFoundError:
    scam_reports = {}
    with open("scam_reports.json", "w") as f:
        json.dump(scam_reports, f, indent=2)
    print("scam_reports.json not found. Initializing scam_reports as empty and creating the file.")

scam_reports.json not found. Initializing verified_data as empty.
scam_reports.json not found. Initializing scam_reports as empty and creating the file.


In [19]:
def lookup_number(phone):
    for entry in verified_data:
        if entry["number"] == phone:
            return entry
    return {"number": phone, "status": "Unknown", "org": ""}

def add_report(number, reason):
    with open("scam_reports.json", "r") as f:
        reports = json.load(f)

    if number in reports:
        reports[number].append(reason)
    else:
        reports[number] = [reason]

    with open("scam_reports.json", "w") as f:
        json.dump(reports, f, indent=2)

def get_risk_level(phone):
    with open("scam_reports.json", "r") as f:
        reports = json.load(f)

    count = len(reports.get(phone, []))

    if count == 0:
        return "Low"
    elif count <= 3:
        return "Medium"
    else:
        return "High"


In [17]:
# UI Elements
phone_input = widgets.Text(
    description='📞 Number:',
    placeholder='Enter phone number',
    layout=widgets.Layout(width='300px')
)
check_button = widgets.Button(description='Check Caller ✅', button_style='success')
dash_button = widgets.Button(description='📊 View Scam Dashboard', button_style='primary')
main_output = widgets.Output()

def show_report_section(phone):
    report_input = widgets.Text(
        placeholder="Enter reason for report",
        layout=widgets.Layout(width='300px')
    )
    submit_button = widgets.Button(description='Submit Report 🚨', button_style='danger')
    report_output = widgets.Output()

    def on_submit_clicked(b):
        reason = report_input.value.strip()
        if reason:
            add_report(phone, reason)
            report_input.value = ""
            with report_output:
                clear_output()
                print("✅ Report submitted.")
                print(f"🛡️ Updated Risk Level: {get_risk_level(phone)}")

    submit_button.on_click(on_submit_clicked)
    display(widgets.HTML("<b>📣 Report Scam</b><br>"))
    display(report_input, submit_button, report_output)

# Event handler for check_button
def on_check_clicked(b):
    with main_output:
        clear_output()
        phone = phone_input.value.strip()
        info = lookup_number(phone)

        print(f"\n📞 Number: {info['number']}")
        print(f"🔎 Status: {info['status']}")
        if info["org"]:
            print(f"🏢 Organization: {info['org']}")
        print(f"🛡️ Current Risk: {get_risk_level(phone)}\n")

        show_report_section(phone)

# Event handler for dash_button
def on_dash_clicked(b):
    with main_output:
        clear_output()
        print("📊 Scam Dashboard:\n")
        show_dashboard()

# Attach the handlers
check_button.on_click(on_check_clicked)
dash_button.on_click(on_dash_clicked)

# Display everything
display(widgets.VBox([
    widgets.HTML("<h3>📱 EduSecure — Caller Checker</h3>"),
    phone_input,
    widgets.HBox([check_button, dash_button]),
    main_output
]))
